## Real Reviews
### Optimizing Business Reviews Using NLP

In [1]:
import json
import pandas as pd
import nltk
import codecs
import sklearn.metrics
import pickle
import numpy as np
import xgboost as xgb
import sklearn.feature_extraction.text
import sklearn.naive_bayes
import sklearn.cross_validation
from scipy.sparse import coo_matrix, vstack, hstack
from langdetect import detect
from yelpapi import YelpAPI
from pprint import pprint
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Read the Pickled DataFrame
df = pd.read_pickle('data/pandas_200k.pkl')

In [167]:
# Read the Yelp reviews dataset into a DataFrame
# Using 200,000 records to build model
# Use 'langdetect' module to only include English reviews
#  for our training set
# with codecs.open('data/yelp_academic_dataset_review.json', 'r', encoding='utf-8') as f:
#     d = [json.loads(f.readline().strip()) for x in range(200000) if detect(json.loads(f.readline().strip())['text'])=="en"]

In [168]:
# Create the DataFrame for training
# df = pd.DataFrame(d)
# len(df)

188096

In [169]:
# Pickle the DataFrame
# df.to_pickle('data/pandas_200k.pkl')

In [3]:
df

,business_id,cool,date,funny,review_id,stars,text,type,useful,user_id
0,2aFiy99vNLklCx3T_tGS9A,0,2010-12-29,0,pXbbIgOXvLuTi_SPs1hQEQ,5,After being on the phone with Verizon Wireless...,review,1,bQ7fQq1otn9hKX-gXRsrgA
1,2LfIuF3_sX6uwe-IR-P0jQ,1,2014-07-14,0,GP6YEearUWrzPtQYSF1vVg,5,Highly recommended. Went in yesterday looking ...,review,0,aW3ix1KNZAvoM8q-WghA3Q
2,2LfIuF3_sX6uwe-IR-P0jQ,1,2013-04-28,0,Uf1Ki1yyH_JDKhLvn2e4FQ,5,What a great place! Modern on Melrose has amaz...,review,2,bgl3j8yJcRO-00NkUYsXGQ
3,2LfIuF3_sX6uwe-IR-P0jQ,0,2012-09-18,0,bRvdVt88MJ_YMTlLbjDLxQ,5,This place is a great for those vintage/mid ce...,review,2,GJ7PTY7huYORFKKg3db3Gw
4,2LfIuF3_sX6uwe-IR-P0jQ,0,2015-04-05,0,LkP1l7sZIwOV6IKNLqQp_A,5,"Great items at a good price. Helpful, easy to...",review,0,UU0nHQtHPMAfLidk8tOHTg
5,2LfIuF3_sX6uwe-IR-P0jQ,1,2014-08-23,1,dJguzIfNKtGkKk8ryOPdfQ,5,I'm a local Real Estate agent and have fallen ...,review,1,OvD92wp0-uuFoGLBymwfKQ
6,2LfIuF3_sX6uwe-IR-P0jQ,0,2014-08-05,0,cNA3oGTmmeRvt10vrvlJOQ,5,Walked into the store and stuck up a conversat...,review,1,AziQIgYIAY6uVw1k8sbtTw
7,2LfIuF3_sX6uwe-IR-P0jQ,2,2013-11-17,3,klVqdjXPM2aM8Y0RsDCPoA,1,"Unfortunately, Yelp does not allow negative st...",review,4,wnzfuir72IZFg5RAPOwWCQ
8,2LfIuF3_sX6uwe-IR-P0jQ,0,2014-08-23,0,k4xighfFxKv5H-Eic28MNQ,5,I stumbled upon Modern on Melrose earlier in t...,review,0,oKZiDLm0D1PIm1-vRtvONA
9,0czfEgv9KAD4VlIa7ANPWQ,1,2009-04-10,2,_a7Zu2ZSEGO4bl2gvu7OtQ,5,"I love Mint, and even though I'm a guy and the...",review,2,jhhHm3Vk9ZlP21WdY_5R0w


## Look Up Business Data Using Yelp API

In [170]:
# How many unique Businesses?
len(df['business_id'].unique())

33288

In [38]:
# Initialize the Yelp API
yelp_id = ""
yelp_secret = ""

with open('data/pvt.csv') as f:
    for line in f:
        l = line.strip().split(',')
        if l[0] == "yelp_id":
            yelp_id = l[1]
        if l[0] == "yelp_secret":
            yelp_secret = l[1]

yelp_api = YelpAPI(yelp_id, yelp_secret)

In [89]:
# Choose a business and store the response from API
bid = df['business_id'].sample(1).iloc[0]
response = yelp_api.business_query(id=bid)
print bid
print response

KuHNBKXBCk3XXNJlZW7jRQ
{u'is_claimed': False, u'rating': 3.5, u'is_closed': True, u'review_count': 3, u'name': u'Half Moon Sports Grill', u'transactions': [], u'url': u'https://www.yelp.com/biz/half-moon-sports-grill-tempe?adjust_creative=fb-xs7dYOAKPmOWg2AKm6Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=fb-xs7dYOAKPmOWg2AKm6Q', u'price': u'$$$', u'photos': [], u'phone': u'+14804569400', u'image_url': u'', u'coordinates': {u'latitude': 33.348099, u'longitude': -111.943092}, u'display_phone': u'(480) 456-9400', u'id': u'half-moon-sports-grill-tempe', u'categories': [{u'alias': u'tradamerican', u'title': u'American (Traditional)'}, {u'alias': u'amusementparks', u'title': u'Amusement Parks'}], u'location': {u'cross_streets': u'', u'city': u'Tempe', u'display_address': [u'325 W Elliot Rd', u'Ste 104', u'Tempe, AZ 85284'], u'country': u'US', u'address2': u'Ste 104', u'address3': u'', u'state': u'AZ', u'address1': u'325 W Elliot Rd', u'zip_code': u'85284'}}


In [76]:
# Variables from 'response' that we will need
city = response['location']['city']
country = response['location']['country']
name = response['name']

In [9]:
print(city)
print(country)
print(name)

Mississauga
CA
Cyclepath Mississauga


## Sentiment Analysis

In [25]:
# Conduct sentiment analysis on reviews
sentiments = [TextBlob(i).sentiment for i in df['text']]

In [26]:
# Add columns for polarity and subjectivity ranking
df['polarity'] = [i[0] for i in sentiments]
df['subjectivity'] = [i[1] for i in sentiments]

In [27]:
# Add column for review wordcount
df['wc'] = [len(x.split()) for x in df['text']]

In [28]:
df[df['polarity']<-.6].iloc[15]

business_id                                3GvyxZvtRe_dwIgHl4rKkw
cool                                                            0
date                                                   2013-01-03
funny                                                           1
review_id                                  J_5asg4U29UvtydGHOzHdA
stars                                                           1
text            Terrible service, no selection... I went to tr...
type                                                       review
useful                                                          0
user_id                                    rTIVsxvTEm5b94C2th5xQQ
polarity                                                       -1
subjectivity                                                    1
wc                                                             22
Name: 6047, dtype: object

## NLP Model
### Riffle Model (Favorable vs Unfavorable)

In [4]:
# We will only look at 5 star or 1 start reviews for training the model
# This implies that all 5 star reviews are definitely "favorable",
#  while 1 star reviews are definitely "unfavorable"
df_stars = df[(df['stars'] == 5) | (df['stars'] == 1)][['text', 'stars']]

In [5]:
# Breakdown of counts in the new DataFrame
print (df_stars['stars'] == 5).sum()
print (df_stars['stars'] == 1).sum()
print df_stars['stars'].count()

74856
26933
101789


In [6]:
# Binarize the star rankings (1 = Favorable, 0 = Unfavorable)
df_stars['stars'] = df_stars['stars'].map(lambda x: 1 if x == 5 else 0)

In [7]:
# Renumber the index
df_stars.index = range(len(df_stars))

### TFIDF Vectorizer

In [8]:
# Create our TFIDF Vectorizer which will be used to
#  rank the importance of words for our classifier
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,3), strip_accents='unicode', min_df=.005)
vectorizer.fit(df_stars['text'])

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=0.005,
        ngram_range=(1, 3), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents='unicode', sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [9]:
# Transform our sample of reviews into a TFIDF vector
reviews_tfidf = vectorizer.transform(df_stars['text'])

In [10]:
# What are the details of our sparse matrix?
reviews_tfidf

<101789x3842 sparse matrix of type '<type 'numpy.float64'>'
	with 8722574 stored elements in Compressed Sparse Row format>

In [ ]:
# Convert tfidf matrix to DataFrame - This could overload the memory
# reviews_tfidf_arr = reviews_tfidf.toarray()
# reviews_tfidf_df = pd.DataFrame(reviews_tfidf_arr, columns=vectorizer.get_feature_names())

In [ ]:
# reviews_tfidf_df

### XGBoost

In [11]:
# Split sample into Train and Test sets for cross validation
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(reviews_tfidf, df_stars['stars'], test_size=0.3, stratify=df_stars['stars'])

In [14]:
# XGBoost Classifier
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)
param = {'eta':1, 'silent':1, 'objective':'binary:logistic', 'scale_pos_weight': 0.3}
num_round = 50
bst = xgb.train(param, dtrain, num_round)

In [44]:
# XGBoost predictor
xg_prob = bst.predict(dtest)
xg_pred = 1*(xg_prob > 0.5)

In [45]:
# XGBoost Model Performance Metrics
print "f1:", sklearn.metrics.f1_score(y_test, xg_pred)
print "recall:", sklearn.metrics.recall_score(y_test, xg_pred)
print "precision:", sklearn.metrics.precision_score(y_test, xg_pred)
print "accuracy:", sklearn.metrics.accuracy_score(y_test, xg_pred)

f1: 0.95909451047
recall: 0.943313888765
precision: 0.975412100562
accuracy: 0.940825883355


In [68]:
# Train an XGBoost model on the entire dataset
dtrain = xgb.DMatrix(reviews_tfidf, df_stars['stars'])
param = {'eta':1, 'silent':1, 'objective':'binary:logistic', 'scale_pos_weight': 0.3}
num_round = 50
bst = xgb.train(param, dtrain, num_round)

### Multinomial Naive Bayes

In [19]:
%%timeit -n1 -r1
# Multinomial Naive Bayes Cross Validaton on F1 Score
nb = sklearn.naive_bayes.MultinomialNB()
nb_acc = sklearn.cross_validation.cross_val_score(nb, X_train, y_train, cv=2, scoring='f1')
print(nb_acc)

[ 0.95668444  0.95578403]
1 loop, best of 1: 408 ms per loop


In [20]:
# Naive Bayes Train Test Split
nb = sklearn.naive_bayes.MultinomialNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)

In [21]:
# Evaluate performance metrics of train-test split
print "f1:", sklearn.metrics.f1_score(y_test, nb_pred)
print "recall:", sklearn.metrics.recall_score(y_test, nb_pred)
print "precision:", sklearn.metrics.precision_score(y_test, nb_pred)
print "accuracy:", sklearn.metrics.accuracy_score(y_test, nb_pred)

f1: 0.957492842625
recall: 0.97546422051
precision: 0.94017167382
accuracy: 0.936306775387


In [55]:
# Train a Naive Bayes model on the entire dataset
nb = sklearn.naive_bayes.MultinomialNB()
nb.fit(reviews_tfidf, df_stars['stars'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
### Random Forests

In [ ]:
%%timeit -n1 -r1
# Random Forests Cross Validaton on F1 Score
forest = RandomForestClassifier()
forest_acc = sklearn.cross_validation.cross_val_score(forest, X_train, y_train, cv=2, scoring='f1')
print(forest_acc)

In [70]:
# Random Forests Train Test Split
forest = RandomForestClassifier(class_weight={0:3}, n_estimators=50)
forest.fit(X_train, y_train)
for_pred = forest.predict(X_test)

In [71]:
# Evaluate performance metrics of train-test split
print "f1:", sklearn.metrics.f1_score(y_test, for_pred)
print "recall:", sklearn.metrics.recall_score(y_test, for_pred)
print "precision:", sklearn.metrics.precision_score(y_test, for_pred)
print "accuracy:", sklearn.metrics.accuracy_score(for_pred, y_test)

f1: 0.957609147155
recall: 0.975241572784
precision: 0.940602989177
accuracy: 0.936503258342


In [72]:
# Evaluate RF performance metrics using a probability threshold
for_probs = forest.predict_proba(X_test)

In [73]:
for_predprobs = 1*(for_probs > .6)[:,1]

In [74]:
print "f1:", sklearn.metrics.f1_score(y_test, for_predprobs)
print "recall:", sklearn.metrics.recall_score(y_test, for_predprobs)
print "precision:", sklearn.metrics.precision_score(y_test, for_predprobs)
print "accuracy:", sklearn.metrics.accuracy_score(y_test, for_predprobs)

f1: 0.956018466389
recall: 0.94518412967
precision: 0.967104064152
accuracy: 0.936044798114


In [75]:
# Train a Random Forests model on the entire dataset
forest = RandomForestClassifier(class_weight={0:3}, n_estimators=50)
forest.fit(reviews_tfidf, df_stars['stars'])

RandomForestClassifier(bootstrap=True, class_weight={0: 3}, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

### Dummy Classifier

In [76]:
# Dummy Classifier
dummy = [1 for x in range(len(y_test))]
print "f1:", sklearn.metrics.f1_score(y_test, dummy)
print "recall:", sklearn.metrics.recall_score(y_test, dummy)
print "precision:", sklearn.metrics.precision_score(y_test, dummy)
print "accuracy:", sklearn.metrics.accuracy_score(y_test, dummy)

f1: 0.847529909046
recall: 1.0
precision: 0.735402953794
accuracy: 0.735402953794


## Save the Models

In [12]:
# Pickle the TFIDF Vectorizer
with open('model/vectorizer_200k_eng.pkl', 'wb') as f:
    pickle.dump(vectorizer, f, protocol=pickle.HIGHEST_PROTOCOL)

In [69]:
# Save the XGBoost model
bst.save_model('model/bst_200k.model')

In [77]:
# Pickle the Naive Bayes model
with open('model/nb_200k_eng.pkl', 'wb') as f:
    pickle.dump(nb, f, protocol=pickle.HIGHEST_PROTOCOL)

In [78]:
# Pickle the Random Forests model
with open('model/forest_200k_eng.pkl', 'wb') as f:
    pickle.dump(forest, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load the TFIDF Vectorizer
with open('model/vectorizer_200k_eng.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

In [ ]:
# Load the XGBoost Model
bst = xgb.Booster({'nthread':4})
bst.load_model('model/bst_200k.model')

In [ ]:
# Load the Random Forests Model
with open('model/nb_200k_eng.pkl', 'rb') as f:
    nb = pickle.load(f)

In [3]:
# Load the Random Forests Model
with open('model/forest_200k_eng.pkl', 'rb') as f:
    forest = pickle.load(f)

## NLP Model
### Reviews Weighted for Helpfulness

In [29]:
# Create weighted DataFrame which will be used for Helpfulness model
# Binarize "Helpful" column
df_weighted = df[['useful', 'polarity', 'subjectivity', 'wc', 'text']]
df_weighted['useful'] = [1 if x >= 2 else 0 for x in df['useful']]

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [30]:
# See the percentage of "Useful" reviews
float(sum(df_weighted['useful']))/len(df_weighted['useful'])

0.2130667318815924

In [31]:
# Create our TFIDF Vectorizer which will be used to
#  rank the importance of words for our classifier
useful_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,3), strip_accents='unicode', min_df=.005)
useful_vectorizer.fit(df_weighted['text'])

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=0.005,
        ngram_range=(1, 3), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents='unicode', sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [32]:
# Transform our sample of reviews into a TFIDF vector
useful_tfidf = vectorizer.transform(df_weighted['text'])

In [33]:
useful_X = df_weighted[['polarity', 'subjectivity', 'wc']]
useful_y = df_weighted['useful']

In [34]:
# Split sample into Train and Test sets for cross validation
# I tried this with TFIDF vector and Polarity/Subjectivity/WordCounts as the features
useful_X_train, useful_X_test, useful_y_train, useful_y_test = sklearn.cross_validation.train_test_split(useful_tfidf, useful_y, test_size=0.3, stratify=useful_y)

In [38]:
# XGBoost Classifier
useful_dtrain = xgb.DMatrix(useful_X_train, useful_y_train)
useful_dtest = xgb.DMatrix(useful_X_test)
param = {'eta':1, 'silent':1, 'objective':'binary:logistic', 'scale_pos_weight': 0.2}
num_round = 25
useful_bst = xgb.train(param, useful_dtrain, num_round)

In [41]:
# XGBoost predictor
useful_xg_prob = useful_bst.predict(useful_dtest)
useful_xg_pred = 1*(useful_xg_prob > 0.5)

In [42]:
# XGBoost Model Performance Metrics
print "f1:", sklearn.metrics.f1_score(useful_y_test, useful_xg_pred)
print "recall:", sklearn.metrics.recall_score(useful_y_test, useful_xg_pred)
print "precision:", sklearn.metrics.precision_score(useful_y_test, useful_xg_pred)
print "accuracy:", sklearn.metrics.accuracy_score(useful_y_test, useful_xg_pred)

f1: 0.0836003051106
recall: 0.0455793063295
precision: 0.504139834407
accuracy: 0.787095287884


### Yeah... That doesn't really work...

## Riffle Model Testing

In [59]:
# Test the model on manually typed text
vec = vectorizer.transform(["The food was just okay."])
man_dtest = xgb.DMatrix(vec)
print bst.predict(man_dtest)
print nb.predict(vec)
# print bst.predict(man_dtest)

[ 0.04751292]
[1]


In [47]:
vec

<1x3842 sparse matrix of type '<type 'numpy.float64'>'
	with 36 stored elements in Compressed Sparse Row format>

### Test the model on a new out of sample test set

In [60]:
# Load dictionary from Yelp JSON file
with codecs.open('data/yelp_academic_dataset_review.json', 'r', encoding='utf-8') as f:
    d_test = []
    for x in range(250000):
        line = f.readline().strip()
        if x <= 200000:
            continue
        else:
            if detect(json.loads(line)['text'])=="en":
                d_test.append(json.loads(line))

LangDetectException: No features in text.

In [64]:
# Create the Pandas DataFrame and preprocess the data
df_test = pd.DataFrame(d_test)
df_stars_test = df_test[(df_test['stars'] == 5) | (df_test['stars'] == 1)][['text', 'stars']]
df_stars_test['stars'] = df_stars_test['stars'].map(lambda x: 1 if x == 5 else 0)

In [65]:
# Create the test set
X_new_test = vectorizer.transform(df_stars_test['text'])
y_new_test = df_stars_test['stars']

In [66]:
# Predict with XGBoost
d_new_test = xgb.DMatrix(X_new_test)
xg_prob_test = bst.predict(d_new_test)
xg_pred_test = 1*(xg_prob_test > 0.5)

In [67]:
# Evaluate performance metrics
print "f1:", sklearn.metrics.f1_score(y_new_test, xg_pred_test)
print "recall:", sklearn.metrics.recall_score(y_new_test, xg_pred_test)
print "precision:", sklearn.metrics.precision_score(y_new_test, xg_pred_test)
print "accuracy:", sklearn.metrics.accuracy_score(y_new_test, xg_pred_test)

f1: 0.97314555323
recall: 0.961918788134
precision: 0.98463747308
accuracy: 0.960141112047
